# フィーチャースケーリング - 課題の回答

対象とするデータが入っているファイルは: 'real_estate_price_size_year.csv'です

データをまずは標準化しましょう

その上で:
-  係数と切片を表示しましょう
-  決定係数と自由度修正済み決定係数を求めましょう
-  決定係数と自由度修正済み決定係数を比較しましょう
-  単回帰分析の場合における決定係数と比較しましょう
-  サイズが750の場合における値段を予測しましょう
-  二つの変数それぞれのp値を求めましょう
-  まとめの表を作成しましょう

## ライブラリのインポート

In [1]:
import sklearn
import pandas as pd
import numpy as np

## データの読み込み

In [2]:
df = pd.read_csv("real_estate_price_size_year.csv")

In [3]:
df

,price,size,year
0,234314.144,643.09,2015
1,228581.528,656.22,2009
2,281626.336,487.29,2018
3,401255.608,1504.75,2015
4,458674.256,1275.46,2009
...,...,...,...
95,252460.400,549.80,2009
96,310522.592,1037.44,2009
97,383635.568,1504.75,2006
98,225145.248,648.29,2015


## 回帰モデルの作成

### 従属変数と独立変数の宣言

In [21]:
x1=df[['size','year']]
y=df['price']
y1 = y.values.reshape(-1,1)

### 入力の標準化

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x1)
x_scaled = scaler.transform(x1)

### 回帰

In [37]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_scaled, y)

LinearRegression()

### 係数の表示

In [38]:
reg.coef_

array([67501.57614152, 13724.39708231])

### 切片の表示

In [39]:
reg.intercept_

292289.4701599997

### 決定係数の計算

In [42]:
reg.score(x_scaled, y)

0.7764803683276793

### 自由度修正済み決定係数の計算

In [43]:
# 以下が自由度修正済み決定係数を求める式です
def adj_r2(x,y):
    r2 = reg.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [46]:
adjusted_rd = adj_r2(x_scaled,y)
print("Adjusted R2:",adjusted_rd)

Adjusted R2: 0.77187171612825


### 結果
これを見る限りは大きな変化がR2と<span style="color: red; ">自由度修正済み決定係数にない</span>

### 決定係数と自由度修正済み決定係数の比較

両者の値にそこまで違いはないことが分かります。つまり、変数を増やすことによるペナルティは小さいことが分かります

### 自由度修正済み決定係数と単回帰の場合の決定係数との比較

両者の値を比較すると、Yearに関する情報は結果に対してほとんど影響を与えていないことが分かります

### 予測
 課題：サイズが750のときの予測をする

In [47]:
new_data = [[750,2009]]
new_data_scaled = scaler.transform(new_data)

In [48]:
reg.predict(new_data_scaled)

array([258330.34465995])

### それぞれの変数のp値の計算

In [49]:
from sklearn.feature_selection import f_regression

In [50]:
f_regression(x_scaled,y)

(array([285.92105192,   0.85525799]), array([8.12763222e-31, 3.57340758e-01]))

In [52]:
p_values = f_regression(x_scaled,y)[1]
p_values

array([8.12763222e-31, 3.57340758e-01])

In [53]:
p_values.round(3)

array([0.   , 0.357])

### まとめ表の作成

In [56]:
reg_summary = pd.DataFrame(data = x1.columns.values, columns=['Features'])
reg_summary ['Coefficients'] = reg.coef_
reg_summary ['p-values'] = p_values.round(3)
reg_summary

,Features,Coefficients,p-values
0,size,67501.576142,0.000
1,year,13724.397082,0.357


### 結果のまとめ
P値を見ると p-value　が非常に大きいこと
さらにさっきの検討からyearは対して決定に際しての影響力が少ない